## Database Assignment #2
#### Author: Lance Barto
#### Principles of Database Systems
#### Dr. Mota
#### October 27, 2019

## Step 1 - Read in the data file

In [1]:
import pandas as pd
import pymysql
import cryptography

ipps_df = pd.read_csv('ipps.csv')

# Display the first few rows of the dataframe to check for integrity after reading the file
ipps_df.head()


,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07,5777.24,4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85,5787.57,4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37,5434.95,4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28,5417.56,4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27,5658.33,4851.44


In [2]:
# Check if we're dealing with any missing values in the dataset
ipps_df.isnull().values.any()


False

## Step 2 - Split the columns as appropriate for 1NF

* First, rename the columns to something that can be used more reasonably

In [3]:
ipps_df.columns = ['drg', 'prov_ID', 'prov_name', 'prov_address', 
                'prov_city', 'prov_state', 'prov_zip', 'hrr', 
                'discharge_count', 'covered_charges', 'total_payments', 
                'medicare_payments']

### Create a new dataframe for the HRR table
* We must be check if we can use the city as a key, or if we need to assign an index

In [4]:
## Create a new dataframe for hrr
hrr_df = pd.DataFrame({'hrr': list(set(ipps_df.hrr))})

## Initial number of unique entries in column
print('Number unique HRR: ', len(set(hrr_df.hrr)))

## Split the columns, drop the original column
hrr_df[['hrr_state', 'hrr_city']] = hrr_df['hrr'].str.split(' - ',expand=True)
hrr_df = hrr_df[['hrr_city', 'hrr_state']]

## Check Length of cities to see if each is unique to a single state,
## If true, then we can use the cities as a key
print('Number unique HRR cities: ', len(set(hrr_df.hrr_city)))


Number unique HRR:  306
Number unique HRR cities:  294


#### Add an hrr_ID

In [5]:
ipps_df['hrr_ID'] = ipps_df.groupby('hrr').ngroup()

#### Remake the table

In [6]:
## Create a new dataframe for hrr
hrr_df = pd.DataFrame({'hrr': list(ipps_df.hrr), 'hrr_ID': list(ipps_df.hrr_ID)})

## Split the columns, drop the original column
hrr_df[['hrr_state', 'hrr_city']] = hrr_df['hrr'].str.split(' - ',expand=True)
hrr_df = hrr_df[['hrr_ID', 'hrr_city', 'hrr_state']]

## Drop duplicates
hrr_df = hrr_df.drop_duplicates()

## Check data integrity
hrr_df.head()


,hrr_ID,hrr_city,hrr_state
0,2,Dothan,AL
1,1,Birmingham,AL
5,5,Montgomery,AL
8,3,Huntsville,AL
14,4,Mobile,AL


### Drop the necessary columns from the original dataframe
* First, need to double check that the states line up before dropping both states columns


In [7]:
## Split the column in the original dataframe
ipps_df[['hrr_state', 'hrr_city']] = ipps_df['hrr'].str.split(' - ', expand=True)

## See if all states are matching by True counts
sum(ipps_df.hrr_state == ipps_df.prov_state) == len(ipps_df)

## Print an example where prov_state != hrr_state
print(ipps_df.iloc[163056])


drg                     948 - SIGNS & SYMPTOMS W/O MCC
prov_ID                                         530002
prov_name            CAMPBELL COUNTY MEMORIAL HOSPITAL
prov_address                    501 SOUTH BURMA AVENUE
prov_city                                     GILLETTE
prov_state                                          WY
prov_zip                                         82716
hrr                                      MT - Billings
discharge_count                                     12
covered_charges                                13756.5
total_payments                                 6889.33
medicare_payments                               5673.5
hrr_ID                                             167
hrr_state                                           MT
hrr_city                                      Billings
Name: 163056, dtype: object


#### Here, the two states columns do not match, therefore, we cannot drop the prov_state column
* Note: All HRR information can be obtained with a join of the providers relation, so we can drop all hrr columns eventually, but must retain the primary key 'hrr_city' for now

In [8]:
## Drop hrr, hrr_state
ipps_df = ipps_df.drop(['hrr', 'hrr_state'], axis=1)

ipps_df.head()


,drg,prov_ID,prov_name,prov_address,prov_city,prov_state,prov_zip,discharge_count,covered_charges,total_payments,medicare_payments,hrr_ID,hrr_city
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,91,32963.07,5777.24,4763.73,2,Dothan
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,14,15131.85,5787.57,4976.71,1,Birmingham
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,24,37560.37,5434.95,4453.79,1,Birmingham
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,25,13998.28,5417.56,4129.16,1,Birmingham
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,18,31633.27,5658.33,4851.44,1,Birmingham


### We need to make similar moves for the drg column

In [9]:
## Create new dataframe for drg
drg_df = pd.DataFrame({'drg': list(ipps_df.drg)})

# Drop dupes
drg_df = drg_df.drop_duplicates()

# ## Split the columns, drop the original column
drg_df[['drg_ID', 'drg_desc']] = drg_df['drg'].str.split(' - ',expand=True)
drg_df = drg_df.drop('drg', axis=1)

## Show the first few entries to check for integrity
drg_df.head()


,drg_ID,drg_desc
0,039,EXTRACRANIAL PROCEDURES W/O CC/MCC
1079,057,DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC
1763,069,TRANSIENT ISCHEMIA
2282,064,INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION...
3949,065,INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION...


#### Split the column in the original dataframe, drop the original column, and the new drg_desc column

In [10]:
## Split the columns
ipps_df[['drg_ID', 'drg_desc']] = ipps_df['drg'].str.split(' - ', expand=True)

## Drop the columns
ipps_df = ipps_df.drop(['drg_desc', 'drg'], axis=1)

## Check integrity of data/drops
ipps_df.head()


,prov_ID,prov_name,prov_address,prov_city,prov_state,prov_zip,discharge_count,covered_charges,total_payments,medicare_payments,hrr_ID,hrr_city,drg_ID
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,91,32963.07,5777.24,4763.73,2,Dothan,039
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,14,15131.85,5787.57,4976.71,1,Birmingham,039
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,24,37560.37,5434.95,4453.79,1,Birmingham,039
3,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,25,13998.28,5417.56,4129.16,1,Birmingham,039
4,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,18,31633.27,5658.33,4851.44,1,Birmingham,039


### Next, create a dataframe for the providers and drop columns from original dataframe

In [11]:
## Create the dataframe
prov_df = ipps_df[['prov_ID', 'prov_name', 'prov_address', 'prov_city', 'prov_state',  'prov_zip', 'hrr_ID']]

## Drop the duplicate rows
prov_df = prov_df.drop_duplicates()

## Double check for correct number of entries
print('Number of Providers: ', len(prov_df))

## Double check the integrity of the dataframe
prov_df.head()


Number of Providers:  3337


,prov_ID,prov_name,prov_address,prov_city,prov_state,prov_zip,hrr_ID
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,2
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,1
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,1
3,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,1
4,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,1


#### Now drop everything from original df except for the prov_ID
* Note: Now, we can drop the hrr_city

In [12]:
## Drop the columns
ipps_df = ipps_df.drop(['prov_name', 'prov_address', 'prov_state', 'prov_city', 'prov_zip', 'hrr_ID'], axis=1)

## Check integrity
ipps_df.head()


,prov_ID,discharge_count,covered_charges,total_payments,medicare_payments,hrr_city,drg_ID
0,10001,91,32963.07,5777.24,4763.73,Dothan,039
1,10005,14,15131.85,5787.57,4976.71,Birmingham,039
2,10006,24,37560.37,5434.95,4453.79,Birmingham,039
3,10011,25,13998.28,5417.56,4129.16,Birmingham,039
4,10016,18,31633.27,5658.33,4851.44,Birmingham,039


#### The original dataframe now represents the desired relation for payments
* Rename it for convention's sake

In [13]:
payments_df = ipps_df


#### Let's also add a payment_ID column to use as the primary ID

In [14]:
## Add an index column so each entry has a unique ID
payments_df['payment_ID'] = range(len(payments_df))

payments_df = payments_df[['payment_ID', 'prov_ID', 'drg_ID', 
                           'discharge_count', 'covered_charges',
                          'total_payments', 'medicare_payments']]

### Check if we can breakdown the provider information further by using zip

In [15]:
len(prov_df.prov_zip) == len(set(prov_df.prov_zip))


False

* Looks like there are fewer zips than providers, indicating duplicates which can be broken out.

#### Let's double check that there aren't any zip codes with more than one city/state combination

In [16]:
location_df = prov_df[['prov_zip', 'prov_state', 'prov_city']]

print('Number of Zips: ', len(set(location_df.prov_zip)))
print('Length before drop: ', len(location_df))

location_df = location_df.drop_duplicates()

print('Length after drop: ', len(location_df))


Number of Zips:  3053
Length before drop:  3337
Length after drop:  3063


#### As it turns out, there are some zips associated with more than one city/state combination
* The marginal returns of separating this information to its own relation is not worth the hassle


### Last check: Double check the number of entries are accurate for our 4 relations

In [17]:
print('Number of DRGs: ', len(drg_df))
print('Number of HRRs: ', len(hrr_df))
print('Number of Providers: ', len(prov_df))
print('Number of Charges/Payments: ', len(payments_df))

Number of DRGs:  100
Number of HRRs:  306
Number of Providers:  3337
Number of Charges/Payments:  163065


## Step 3 Create a database connection

In [18]:
server = 'localhost'
database = 'ipps'
user = 'ipps'
password = 'password'

# connects to the database
connection = pymysql.connect(host = server, user = user, password = password, db = database)
if (connection):
    print('Connection to MySQL database', database, 'was successful!')


Connection to MySQL database ipps was successful!


## Step 4 Load the data into the DB

* First, the HRR relation

In [19]:
# new row
cursor = connection.cursor()
sql = 'INSERT INTO HRR VALUES (%s, %s, %s)'

for row in range(len(hrr_df)):
    cursor.execute(sql, tuple([float(hrr_df.iloc[row,0]), str(hrr_df.iloc[row,1]), str(hrr_df.iloc[row,2])]))

connection.commit()

* Next, the DRG relation

In [20]:
# new rows
sql = 'INSERT INTO DRG VALUES (%s, %s)'

for row in range(len(drg_df)):
    cursor.execute(sql, tuple([float(drg_df.iloc[row,0]), str(drg_df.iloc[row,1])]))

connection.commit()

* Next, the Providers relation

In [21]:
# new row
cursor = connection.cursor()
sql = 'INSERT INTO Providers VALUES (%s, %s, %s, %s, %s, %s, %s)'

for row in range(len(prov_df)):
    cursor.execute(sql, tuple([int(prov_df.iloc[row,0]), prov_df.iloc[row,1], 
                               prov_df.iloc[row,2], prov_df.iloc[row,3], 
                               prov_df.iloc[row,4], int(prov_df.iloc[row,5]), 
                               int(prov_df.iloc[row,6])]))

connection.commit()

* Finally, the Payments relation

In [22]:
# new rows
sql = 'INSERT INTO Payments VALUES (%s, %s, %s, %s, %s, %s, %s)'

for row in range(len(payments_df)):
    cursor.execute(sql, tuple([int(payments_df.iloc[row,0]), int(payments_df.iloc[row,1]), 
                               int(payments_df.iloc[row,2]), int(payments_df.iloc[row,3]), 
                               float(payments_df.iloc[row,4]), float(payments_df.iloc[row,5]), 
                               float(payments_df.iloc[row,6])]))

connection.commit()

### Always be sure to terminate the connection

In [23]:
connection.close()